In [76]:
import pandas as pd
import numpy as np

In [77]:
# Load the bradycardia BPM dataset (to be rescaled for tachycardia)
bpm_file = "/home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Tachycardia/CSV_DATA/BPM_8hr_5min.csv"
df = pd.read_csv(bpm_file, header=None)  # Assuming no headers in file

In [78]:
# Define original adult BPM ranges and the new neonatal tachycardia range
adult_normal_range = (60, 100)
adult_tachycardia_range = (80, 180)  # Tachycardia in adults is typically >100 BPM, lower lower bound for more tachycardia
neonatal_normal_range = (100, 180)
neonatal_tachycardia_range = (180, 500)  # Neonatal tachycardia BPM range, raise upper bound for more tachycardia

In [79]:
# Function to rescale values to the new range
def rescale_bpm(data, old_min, old_max, new_min, new_max):
    return ((data - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

In [80]:
# Apply rescaling for tachycardia
df_tachycardia = df.copy()
df_tachycardia[df >= 100] = rescale_bpm(df[df >= 100], *adult_tachycardia_range, *neonatal_tachycardia_range)

# Scale normal BPM values (60-100) -> Neonatal normal range (100-160)
df_tachycardia[df < 80] = rescale_bpm(df[df < 80], *adult_normal_range, *neonatal_normal_range)

In [81]:
# Introduce additional random spikes to tachycardia
tachycardia_mask = df_tachycardia > 160
df_tachycardia[tachycardia_mask] += np.random.uniform(5, 15, size=df_tachycardia[tachycardia_mask].shape)

In [82]:
# # Introduce slight random variation to tachycardia values
# tachy_mask = df_tachycardia > 160
# df_tachycardia[tachy_mask] += np.random.uniform(-5, 5, size=df_tachycardia[tachy_mask].shape)

In [83]:
# Save the rescaled neonatal tachycardia dataset
tachycardia_bpm_file = "/home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Tachycardia/Scaled_CSV_DATA/BPM_Tachycardia_8hr_5min_Neonatal.csv"
df_tachycardia.to_csv(tachycardia_bpm_file, index=False)

print(f"Neonatal Tachycardia BPM data saved: {tachycardia_bpm_file}")

Neonatal Tachycardia BPM data saved: /home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Tachycardia/Scaled_CSV_DATA/BPM_Tachycardia_8hr_5min_Neonatal.csv


In [84]:
import pandas as pd

# Load the neonatal BPM dataset for tachycardia
neonatal_bpm_file = "/home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Tachycardia/Scaled_CSV_DATA/BPM_Tachycardia_8hr_5min_Neonatal.csv"
df_neonatal = pd.read_csv(neonatal_bpm_file, header=None)

# Remove the first row if it contains index-like values
if (df_neonatal.iloc[0] == range(96)).all():  # Check if first row is just index values
    df_neonatal = df_neonatal.iloc[1:].reset_index(drop=True)  # Remove it

# Convert to numeric to ensure proper processing
df_neonatal = df_neonatal.apply(pd.to_numeric, errors="coerce")

# Define neonatal tachycardia thresholds
tachycardia_threshold = 160
severe_tachycardia_threshold = 230

# Identify tachycardia instances (BPM > 160)
tachycardia_instances = df_neonatal[df_neonatal > tachycardia_threshold].stack().reset_index()
tachycardia_instances.columns = ["Row", "Column", "BPM"]

# Identify severe tachycardia instances (BPM > 200)
severe_tachy_instances = tachycardia_instances[tachycardia_instances["BPM"] > severe_tachycardia_threshold]

# Print the detected tachycardia instances
print("\nTachycardia Instances (BPM > 160):")
print(tachycardia_instances)

print("\nSevere Tachycardia Instances (BPM > 230):")
print(severe_tachy_instances)



Tachycardia Instances (BPM > 160):
    Row  Column         BPM
0     3      33  259.420090
1     5       8  271.499277
2     5      15  284.836399
3     5      16  273.012262
4     5      20  259.549496
5     5      21  267.222555
6     5      27  267.775158
7     5      71  255.574170
8     5      72  267.235640
9     5      75  261.211893
10    9       7  268.168000
11    9       8  281.050984
12    9      15  292.941471
13    9      16  271.667271
14    9      20  265.320113
15    9      21  255.907850
16    9      27  258.319940
17    9      51  251.954009
18    9      69  258.839784
19    9      71  254.381071
20    9      72  279.023223
21    9      75  270.921059

Severe Tachycardia Instances (BPM > 230):
    Row  Column         BPM
0     3      33  259.420090
1     5       8  271.499277
2     5      15  284.836399
3     5      16  273.012262
4     5      20  259.549496
5     5      21  267.222555
6     5      27  267.775158
7     5      71  255.574170
8     5      72  267.2356